# **FOUNDATION OF COMPUTER SCIENCE PROJECT**

Il progetto riguarda lo studio del dataset [Kiva](https://drive.google.com/file/d/1-tJtnIbo1Rt-F1XfoWGVkmBXiI-ciuRx/view?usp=sharing). In particolare la ricerca di soluzioni ottimali alle richieste consultabili alla [pagina](http://gianluca.dellavedova.org/foundationsCS/2019-project) del corso di Foundation of Computer Science presso Università degli Studi di Milano - Bicocca.

---

**1.** Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [1]:
import pandas as pd
import itertools as it
loans_lenders=pd.read_csv('/content/drive/My Drive/loans_lenders.csv')

In [2]:
%%time
flatten = lambda l: [item for sublist in l for item in sublist]

l=[]
l1=[]
for i in range(0,len(loans_lenders)):
    x=loans_lenders.iloc[i,1].split(', ')
    l.append(x)
    k=len(x)
    l1.append(list(it.repeat(loans_lenders.iloc[i,0],k)))
    
df1=pd.DataFrame({'loan': flatten(l1),'lender':flatten(l)})

CPU times: user 47.6 s, sys: 3.1 s, total: 50.7 s
Wall time: 50.8 s


In [3]:
df1

,loan,lender
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
...,...,...
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175


**2.** For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.

In [4]:
loans=pd.read_csv('/content/drive/My Drive/loans.csv')

In [5]:
%%time
time_delta=pd.to_datetime(loans['planned_expiration_time'])-pd.to_datetime(loans['disburse_time'])
loans['duration']=time_delta.dt.days
df2=loans[['loan_id','disburse_time','planned_expiration_time','duration']]

CPU times: user 1 s, sys: 3.06 ms, total: 1 s
Wall time: 1.03 s


In [6]:
df2

,loan_id,disburse_time,planned_expiration_time,duration
0,657307,2013-12-22 08:00:00.000 +0000,2014-02-14 03:30:06.000 +0000,53.0
1,657259,2013-12-20 08:00:00.000 +0000,2014-03-26 22:25:07.000 +0000,96.0
2,658010,2014-01-09 08:00:00.000 +0000,2014-02-15 21:10:05.000 +0000,37.0
3,659347,2014-01-17 08:00:00.000 +0000,2014-02-21 03:10:02.000 +0000,34.0
4,656933,2013-12-17 08:00:00.000 +0000,2014-02-13 06:10:02.000 +0000,57.0
...,...,...,...,...
1419602,988180,2015-11-23 08:00:00.000 +0000,2016-01-02 01:00:03.000 +0000,39.0
1419603,988213,2015-11-24 08:00:00.000 +0000,2016-01-02 16:40:07.000 +0000,39.0
1419604,989109,2015-11-13 08:00:00.000 +0000,2016-01-03 22:20:04.000 +0000,51.0
1419605,989143,2015-11-03 08:00:00.000 +0000,2016-01-05 08:50:02.000 +0000,63.0


**3.** Find the lenders that have funded at least twice.

In [7]:
%%time
df1_dup=df1.groupby('lender').count()
df3=df1_dup.loc[df1_dup['loan']>1,:]

CPU times: user 13.8 s, sys: 630 ms, total: 14.4 s
Wall time: 14.4 s


In [8]:
df3.sort_values(by=['loan'],ascending=True)

,loan
lender,
jennifer27158118,2
stacey2336,2
laura8785,2
laura87813242,2
laura8777,2
...,...
themissionbeltco,81434
nms,104314
gmct,128159


**4.** For each country, compute how many loans have involved that country as borrowers

In [9]:
%%time
df4=loans.groupby('country_name')['loan_id'].count()

CPU times: user 93.2 ms, sys: 1.87 ms, total: 95.1 ms
Wall time: 95.6 ms


In [10]:
df4.to_frame()

,loan_id
country_name,
Afghanistan,2337
Albania,3075
Armenia,13952
Azerbaijan,10172
Belize,218
...,...
Vietnam,21839
Virgin Islands,2
Yemen,4206


**5.** For each country, compute the overall amount of money borrowed.

In [11]:
%%time
df5=loans.groupby('country_name')['loan_amount'].sum().sort_values(ascending=False)

CPU times: user 89.4 ms, sys: 1.96 ms, total: 91.4 ms
Wall time: 96.2 ms


In [12]:
df5.to_frame()

,loan_amount
country_name,
Philippines,97984600.0
Peru,79437775.0
Kenya,66735975.0
Paraguay,53964700.0
Cambodia,51613525.0
...,...
Virgin Islands,10000.0
Vanuatu,9250.0
Botswana,8000.0


**6.** Like the previous point, but expressed as a percentage of the overall amount lent.

In [13]:
%%time
tot=loans['loan_amount'].sum()
df6=loans.groupby('country_name')['loan_amount'].apply(lambda x: (x.sum()/tot)*100).sort_values(ascending=False)

CPU times: user 204 ms, sys: 6.09 ms, total: 210 ms
Wall time: 214 ms


In [14]:
df6.to_frame()

,loan_amount
country_name,
Philippines,8.293678
Peru,6.723825
Kenya,5.648711
Paraguay,4.567716
Cambodia,4.368706
...,...
Virgin Islands,0.000846
Vanuatu,0.000783
Botswana,0.000677


**7.** Like the three previous points, but split for each year (with respect to disburse time).

In [15]:
%%time
loans['year']=pd.to_datetime(loans['disburse_time']).dt.year
df7=(loans.groupby(['year','country_name'])['loan_amount'].sum()/loans.groupby(['year'])['loan_amount'].sum())*100

CPU times: user 642 ms, sys: 10.1 ms, total: 652 ms
Wall time: 672 ms


In [16]:
df7.to_frame()

loan_amount
year   country_name             
2005.0 Bulgaria         1.798736
       Cambodia         1.579971
       Ecuador          4.861449
       Gaza             4.861449
       Honduras        32.377248
...                          ...
2018.0 Tanzania         1.444408
       Thailand         2.676700
       Uganda           2.196914
       Uruguay          0.808060
       Zambia           1.411581

[748 rows x 1 columns]

**8.** For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [17]:
%%time
df1_loan=df1.groupby('loan').count()
df1_loan=df1_loan.reset_index()

loans_ridotto=loans[['loan_id','loan_amount']]
loans_merge=pd.merge(df1_loan,loans_ridotto,left_on='loan',right_on='loan_id')[['loan','lender','loan_amount']]
loans_merge['amount']=round(loans_merge['loan_amount']/loans_merge['lender'],2)

df8_merge=pd.merge(df1,loans_merge[['loan','amount']])
df8=df8_merge.groupby('lender').sum()['amount']

CPU times: user 20.4 s, sys: 894 ms, total: 21.3 s
Wall time: 21.3 s


In [18]:
#step_1
loans_merge.head()

,loan,lender,loan_amount,amount
0,84,3,500.0,166.67
1,85,2,500.0,250.00
2,86,3,500.0,166.67
3,88,3,300.0,100.00
4,89,4,500.0,125.00


In [19]:
#step_2
df8_merge.head()

,loan,lender,amount
0,483693,muc888,30.62
1,483693,sam4326,30.62
2,483693,camaran3922,30.62
3,483693,lachheb1865,30.62
4,483693,rebecca3499,30.62


In [20]:
#risultato finale
df8.to_frame()

,amount
lender,
000,1764.26
00000,1380.68
0002,2472.54
00mike00,52.63
0101craign0101,2623.50
...,...
zzmcfate,66113.04
zzpaghetti9994,51.02
zzrvmf8538,576.97


**9.** For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [21]:
lenders=pd.read_csv('/content/drive/My Drive/lenders.csv')
country_stats=pd.read_csv('/content/drive/My Drive/country_stats.csv')

In [22]:
%%time
lenders_1=lenders[['permanent_name','country_code']].dropna()
lenders_merge=pd.merge(country_stats[['country_code','country_name']],lenders_1,on='country_code')
df8_1=pd.DataFrame(df8).reset_index()
lenders_merge_1=pd.merge(lenders_merge,df8_1,left_on='permanent_name',right_on='lender')
group_1=lenders_merge_1[['country_name','amount']].groupby('country_name').sum()

loans_1=df8_merge[['loan','amount']].groupby('loan').sum()
loans_merge=pd.merge(loans_1,loans[['loan_id','country_name']],left_on='loan',right_on='loan_id')
group_2=loans_merge[['country_name','amount']].groupby('country_name').sum()

df9 = group_1.merge(group_2, left_index=True, right_index=True)
df9['amount_difference']=df9['amount_y']-df9['amount_x']

CPU times: user 5.33 s, sys: 469 ms, total: 5.79 s
Wall time: 5.8 s


In [23]:
#step_1
group_1.head()

,amount
country_name,
Afghanistan,107448.60
Albania,8033.27
Algeria,15315.87
Angola,34805.58
Argentina,295252.01


In [24]:
#step_2
group_2.head()

,amount
country_name,
Afghanistan,1966915.50
Albania,4283756.54
Armenia,22901005.77
Azerbaijan,14749023.59
Belize,150175.35


In [25]:
#risultato finale
df9['amount_difference'].to_frame()

,amount_difference
country_name,
Afghanistan,1.859467e+06
Albania,4.275723e+06
Armenia,2.287729e+07
Azerbaijan,1.474031e+07
Belize,1.279613e+05
...,...
United States,-5.149399e+08
Uruguay,-8.840313e+04
Yemen,3.423545e+06


**10.** Which country has the highest ratio between the difference computed at the previous point and the population?

In [26]:
%%time
df10=pd.merge(df9,country_stats,left_index=True,right_on='country_name')[['country_name','amount_difference','population']]
df10['ratio']=df10['amount_difference']/df10['population']
df10_max=df10[df10['ratio']==max(df10['ratio'])]

CPU times: user 10 ms, sys: 2 ms, total: 12 ms
Wall time: 12.5 ms


In [27]:
df10_sorted=df10.sort_values(by=['ratio'],ascending=False)
df10_sorted.head()

,country_name,amount_difference,population,ratio
171,Samoa,1.218253e+07,196440,62.016565
109,Armenia,2.287729e+07,2930450,7.806750
93,Paraguay,5.303712e+07,6811297,7.786640
82,El Salvador,4.146874e+07,6377853,6.501990
119,Mongolia,1.527825e+07,3075647,4.967491


**11.** Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [28]:
%%time
country_stats['population_over_poverty_line']=(1-(country_stats['population_below_poverty_line']/100))*country_stats['population']
country_stats_poverty=country_stats[['country_name','population_over_poverty_line']]
df11=pd.merge(df9,country_stats_poverty,left_index=True,right_on='country_name')[['country_name','amount_difference','population_over_poverty_line']]
df11['ratio_over_poverty']=df11['amount_difference']/df11['population_over_poverty_line']

CPU times: user 11.4 ms, sys: 28 µs, total: 11.4 ms
Wall time: 11 ms


In [29]:
df11.loc[df11['ratio_over_poverty'].idxmax()]

country_name                        Armenia
amount_difference               2.28773e+07
population_over_poverty_line    1.99271e+06
ratio_over_poverty                  11.4805
Name: 109, dtype: object

12.For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. 
For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [30]:
%%time
loans_ridotto_2=loans[['loan_id','loan_amount','disburse_time','planned_expiration_time']]
loans_ridotto_2['diff_days']=(pd.to_datetime(loans_ridotto_2['planned_expiration_time'])-pd.to_datetime(loans_ridotto_2['disburse_time'])).astype('timedelta64[D]')

loans_ridotto_2['disburse_time']=pd.to_datetime(loans_ridotto_2['disburse_time'])
loans_ridotto_2['planned_expiration_time']=pd.to_datetime(loans_ridotto_2['planned_expiration_time'])

from pandas.tseries import offsets
loans_ridotto_2['x']=loans_ridotto_2['disburse_time'] + offsets.YearEnd()
loans_ridotto_2['y']=loans_ridotto_2['planned_expiration_time'] - offsets.YearBegin()
loans_ridotto_2['diff_year']=(loans_ridotto_2.y-loans_ridotto_2.x).astype('timedelta64[Y]')

subset_1=loans_ridotto_2[loans_ridotto_2['diff_year']<0]
subset_1['year']=subset_1['disburse_time'].dt.year
sub_group_1=subset_1[['year','loan_amount']].groupby('year').sum()
sub_group_1=sub_group_1.reset_index()

subset_2=loans_ridotto_2[loans_ridotto_2['diff_year']>=0]

ls_year=[]
ls_amount=[]
for i in range(0,len(subset_2)):
    diff_y=subset_2.iloc[i,4]
    amount=(subset_2.iloc[i,1])/diff_y
    exp=subset_2.iloc[i,3]
    disb=subset_2.iloc[i,2]
    
    ls_year.append(disb.year)
    ls_amount.append(amount*(subset_2.iloc[i,5]-disb).days)
    
    ls_year.append(exp.year)
    ls_amount.append(amount*(exp-subset_2.iloc[i,6]).days)
    
    while diff_y != 0:
            ls_year.append(exp.year-diff_y)
            ls_amount.append(amount*365)
            diff_y -= 1

year_amount=pd.DataFrame({'year':ls_year,'loan_amount':ls_amount})
sub_group_2=year_amount.groupby('year').sum()
final=pd.merge(sub_group_1,sub_group_2,on='year').set_index('year').sum(axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

CPU times: user 33.7 s, sys: 379 ms, total: 34.1 s
Wall time: 34.1 s


In [31]:
pd.DataFrame({'total_amount':final})

,total_amount
year,
2011,8.490949e+08
2012,9.478558e+08
2013,8.537651e+08
2014,7.285327e+08
2015,5.692740e+08
2016,4.275082e+08
2017,2.800620e+08
2018,5.608753e+06
